<a href="https://colab.research.google.com/github/alonzojp/AI-Chess-Bot/blob/main/%5BAICB%5D_chess_ai_vs_player.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chess
import chess
import time
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [2]:
BE_model = tf.keras.models.load_model('/content/BE_3epochs_64relu.keras')

In [3]:
piece_to_number_array = ['zero', 'p', 'two', 'b', 'n', 'r', 'six', 'seven', 'q', 'k']

def piece_to_number(piece): # lower = black upper = white
  if(piece.islower()): # black piece = negative
    return -1 * piece_to_number_array.index(piece.lower())
  else:
    return piece_to_number_array.index(piece.lower())

def fen_to_array(fen):
  final_array = []

  fen_split = fen.split('/')
  for row in fen_split:
    for column in row:
      if(column.isdigit()):
        for i in range(0, int(column)):
          final_array.append(0)
      else:
        final_array.append(int(piece_to_number(column)))
  return final_array

In [4]:
def display_formatted_board():
  for i in range(0, 50):
    print("")
  display(board)

def make_ai_move():
  # mirrored = chess.Board.mirror(board)
  legal_moves = board.legal_moves
  print(list(legal_moves))
  possible_fens = []
  numerical_fen_representations = []

  for possible_move in legal_moves:
    temp_board = board.copy()
    temp_board.push(possible_move)
    possible_fens.append(temp_board.board_fen().split(" ")[0])
    # display(temp_board)
    # print(temp_board.board_fen().split(" ")[0])
    # print("-" * 50)

  for fen in possible_fens:
    numerical_fen_representations.append(fen_to_array(fen))

  # print(numerical_fen_representations)
  move_evaluations = np.array(BE_model.predict(numerical_fen_representations, verbose = 0) * -1) # multiplying by -1 because black is moving, not white
  best_move_number = list(move_evaluations).index(min(move_evaluations))
  # print(best_move_number)
  # print(possible_fens[best_move_number])

  best_move = list(legal_moves)[best_move_number]
  board.push(best_move)
  move_line.append(best_move)
  # board = chess.Board.mirror(board)

  display_formatted_board()
  # display(chess.Board(possible_fens[best_move_number]))

In [ ]:
board = chess.Board()
move_line = []

display_formatted_board()

while(True):

  # print(move_line)

  # Player's turn:
  print("Please enter a valid move in chess notation: ")
  player_move = input()
  try:
    move = chess.Move.from_uci(player_move)
    if(move not in board.legal_moves):
      raise Exception
    else:
      board.push(move)
      move_line.append(move)
      display_formatted_board()
      time.sleep(1)
      if(board.is_checkmate()):
        print("You won! yippee")
        break
    # AI's Turn:
    make_ai_move()
    # board.push(chess.Move.from_uci("a7a6")) # how to push AI move correctly
    # display_formatted_board()

    if(board.is_checkmate()):
      print("You lost! skull emoji moment")
      break
  except:
    display_formatted_board()
    print("Invalid move, please try again!")



In [ ]:
board = chess.Board()
display(board)
board.push_san("e4")
display(board) # shows white move
board = chess.Board.mirror(board)
display(board)


board.push_san("Nh3")
display(board)

board = chess.Board.mirror(board)
display(board)
